In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

class BM25:

    def __init__(self, documents, k1=1.5, b=0.75):
        self.k1 = k1
        self.b = b
        self.documents = documents
        self.N = len(documents)
        self.doc_lengths = [len(doc.split()) for doc in documents]
        self.avgdl = np.mean(self.doc_lengths)
        self.doc_freqs = self._compute_doc_freqs()
    
    def _compute_doc_freqs(self):
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(self.documents)
        self.vocabulary = vectorizer.get_feature_names_out()
        doc_freqs = np.array((X > 0).sum(axis=0)).flatten()
        return dict(zip(self.vocabulary, doc_freqs))
    
    def _idf(self, term):
        df = self.doc_freqs.get(term, 0)
        return np.log((self.N - df + 0.5) / (df + 0.5) + 1.0)
    
    def _score(self, query, doc_index):
        score = 0
        doc = self.documents[doc_index].split()
        doc_len = self.doc_lengths[doc_index]
        
        for term in query.split():
            if term in self.doc_freqs:
                idf = self._idf(term)
                tf = doc.count(term)
                score += idf * (tf * (self.k1 + 1)) / (tf + self.k1 * (1 - self.b + self.b * (doc_len / self.avgdl)))
        
        return score
    
    def search(self, query):
        scores = [self._score(query, i) for i in range(self.N)]
        return np.argsort(scores)[::-1]  # Sort by descending scores

# Load the 20 Newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

# Create an instance of BM25
bm25 = BM25(documents)

# Query example
query = "space exploration"
results = bm25.search(query)

# Display results
for index in results[:5]:  # Show top 5 results
    print(f"Document: {documents[index][:500]}...")  # Print the first 500 characters of the document
    print(f"Score: {bm25._score(query, index)}")
    print()

Document: We are not at the end of the Space Age, but only at the end of Its
beginning.

That space exploration is no longer a driver for technical innovation,
or a focus of American cultural attention is certainly debatable; however,
technical developments in other quarters will always be examined for
possible applications in the space area and we can look forward to
many innovations that might enhance the capabilities and lower the
cost of future space operations. 

The Dream is Alive and Well....
Score: 14.515338105394955

Document: There is an interesting opinion piece in the business section of today's
LA Times (Thursday April 15, 1993, p. D1).  I thought I'd post it to
stir up some flame wars - I mean reasoned debate.  Let me preface it by
saying that I largely agree that the "Space Age" in the romantic sense
of several decades ago is over, and that projects like the space station
miss the point at this time.  Reading, for example, "What's New" -
the weekly physics update we get 